In [11]:
"""
1. All sgRNA where blasted to MANE transcript database. 
2. Top 5 hits for each sequence were stored
3. Ensembl Gene ID for all the transcripts where retrieved uising bioMart
4. Select threshold 
"""
import pandas as pd
import biomart

In [13]:
df = pd.read_csv(
    r"/home/cfvall/output/result.tsv",
    sep='\t',
    header=None,
    names=["id", "transcript_id", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "sstrand"]
)
df['transcript_id'] = df['transcript_id'].str.split('.').str[0]
df['library_geneSymbol'] = df["id"].str.split('_').str[0]


In [3]:
def get_ensembl_mappings(dataset):                                   
    # Set up connection to server                                               
    mart = dataset                           
                                                                                
    # List the types of data we want                                            
    attributes = ['ensembl_transcript_id', 'hgnc_symbol', 
                  'ensembl_gene_id', 'entrezgene_id']
                                                                                
    # Get the mapping between the attributes                                    
    response = mart.search({'attributes': attributes})                          
    data = response.raw.data.decode('ascii')                                    
                                                                                
    # Create a list to hold our data rows                                             
    rows_list = []                                            
    
    # Iterate over each line in the data and store it in our list
    for line in data.split('\n'):                                              
        if line:  # check that line is not empty
            line_data = line.split('\t')                                                 
            row_dict = {
                'transcript_id': line_data[0],
                'gene_symbol': line_data[1],
                'ensembl_gene': line_data[2],
                'entrezgene_id': line_data[3] if len(line_data) > 3 else None
            }
            rows_list.append(row_dict)
    
    # Convert our list of dicts into a dataframe                                 
    df_out = pd.DataFrame(rows_list)                                                                                
    return df_out



In [4]:
server = biomart.BiomartServer('http://www.ensembl.org/biomart')

# Select the appropriate dataset
dataset = server.datasets['hsapiens_gene_ensembl']

biomartDB = get_ensembl_mappings(dataset)

In [24]:
#print retrievable attributes
# dataset.attributes

In [14]:
df_merged = pd.merge(left=df, right=biomartDB, on='transcript_id' , how='left')

In [15]:
df_merged['geneSymbol_match'] = df_merged['library_geneSymbol'] == df_merged['gene_symbol']

In [7]:
df_merged_subset = df_merged[df_merged['evalue'] < 0.1]

In [20]:
df_merged['index'] = df_merged["id"].str.split('_').str[-1]

In [21]:
df_merged.to_csv('/home/cfvall/output/blast_results.tsv' , header=True, index=False, sep='\t')